# **Experimentation V2**

### **Create Paper Data**

In [5]:
# Import the paperscraper module
from paperscraper import paperscraper

# Create an instance of the class
scraper = paperscraper('2024-10-04')

# Get sections of AIAYN paper using html_subdivide function
AIAYN_sections = scraper.html_subdivide('https://arxiv.org/html/1706.03762v7')